In [1]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install langdetect
!pip install polars

  Cloning https://github.com/boudinfl/pke.git to c:\users\xavmo\appdata\local\temp\pip-req-build-i1zp42f6
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63

  Running command git clone -q https://github.com/boudinfl/pke.git 'C:\Users\xavmo\AppData\Local\Temp\pip-req-build-i1zp42f6'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.



  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6161142 sha256=3ce9f69aa1d4e248daa516120f6119948394f6501b2427d7697449fca896b064
  Stored in directory: C:\Users\xavmo\AppData\Local\Temp\pip-ephem-wheel-cache-bmrn1f1u\wheels\d5\46\97\85535b5b449f70b6a3c8d1138ce8587345876891e25bfe7954
Successfully built pke
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4
  Attempting uninstall: smart-open
    Found existing installation: smart-open 5.1.0
    Uninstalling smart-open-5.1.0:
      Successfully uninstalled smart

In [1]:
# Librairies nécessaires
import numpy as np
import requests
import pandas as pd
import polars as pl
from langdetect import detect
import json
import tqdm
from tqdm import trange

import pke

import seaborn as sns
import matplotlib.pyplot as plt

## Importation d'un corpus : 

Sorbonne Université à tous son corpus de publications principalement sur HAL.  
Cependant, suite à une annonce de partenariat avec OpenAlex et allant de plus en plus sur cette bibliothèque gratuite et open source, le choix d'aller chercher dans un premier temps mon corpus dessus est donc le plus pertinant.  
Dans un premier temps, il nous faut donc comparer la différence des mots clés utilisés dans HAL et OpenAlex via le travail de Nacef effectué sur le github suivant (https://github.com/obtic-sorbonne/keywords/blob/main/TER_Keyword_Extraction_sur_BDD_HAL.ipynb).  
Regardons un premier temps quelles sont les métadonnées des publications sur HAl et OA en prenant pour exemple une publication de Stephane Le Crom

In [13]:
import requests
import json

hal_id = 'hal-00122816'
url = f"https://api.archives-ouvertes.fr/search/?q=halId_s:{hal_id}&wt=json"

# Faire la requête à l'API
if response.status_code == 200:
    
    # Conversion de la réponse en JSON
    data = json.loads(response.text)
    # Prendre le premier document (s'il y en a plusieurs)
    doc = data['response']['docs'][0]
    print(doc)
    # Vérifier s'il y a des documents dans la réponse
    if data['response']['numFound'] > 0:
        
        # Afficher les noms de toutes les métadonnées
        for key in doc.keys():
            print(key)
    else:
        print("Aucune publication trouvée avec cet identifiant HAL.")
else:
    print(f"Erreur lors de la requête : {response.status_code}")

{'docid': '122816', 'label_s': 'Marika Kapsimali, Stéphane Le Crom, Philippe Vernier. A natural history of vertebrate dopamine receptors.. Anita Sidhu, Marc Laruelle, Philippe Vernier. Dopamine Receptors and Transporters, Marcel Dekker Inc, pp.1-45, 2003. &#x27E8;hal-00122816&#x27E9;', 'uri_s': 'https://hal.science/hal-00122816'}
docid
label_s
uri_s


In [ ]:
# test a deux lvl pour keywords -> mots clés exacts similaires ou etymologiquement similaire (via du LLM ??)
#                               -> comparaison par topic de ses mots clés d'open alex (Comparaison avec les autres topics??)
#                               -> comment calculer la similarité?

In [23]:
# Script de Waly modifié.
# Liste des noms d'auteurs que vous souhaitez rechercher
auteurs = [
    "Anne-Virginie Salsac",
    "Dan Istrate",
    "Eric Leclerc",
    "Julien \"Le Duigou\"",
    "Marie-Christine \"Ho Ba Tho\"",
    "Sofiane Boudaoud",
    "Glenn Roe",
    "Motasem ALRAHABI",
    "Arnaud Latil",
    "Christian NERI",
    "Clément Mabi",
    "David Flacher",
    "M. Shawky",
    "Serge Bouchardon",
    "Harry Sokol",
    "Bérangère Bihan-Avalle",
    "Caroline Marti",
    "Laurent Petit",
    "Pierre-Carl Langlais",
    "David Klatzmann",
    "Raphael Gavazzi",
    "Benjamin Wandelt",
    "christophe pichon",
    "Guilhem Lavaux",
    "Henry Joy McCracken",
    "Kumiko Kotera",
    "Yohan Dubois",
    "A. Marco Saitta",
    "Dirk Stratmann",
    "Guillaume Ferlat",
    "Slavica Jonic",
    "Alex Chin",
    "Fabrice Carrat",
    "Pierre-Yves Boëlle",
    "Renaud Piarroux",
    "Christophe Guillotel-Nothmann",
    "Jean-Marc Chouvel",
    "Nicolas Obin",
    "Philippe Esling",
    "Alexandre Coninx",
    "Baptiste Caramiaux",
    "Benjamin Piwowarski",
    "Catherine Achard",
    "Catherine Pelachaud",
    "Gilles Bailly",
    "Jérôme Szewczyk",
    "Kevin Bailly",
    "Laure Soulier",
    "Marie-Aude Vitrani",
    "Matthieu Cord",
    "Mehdi Khamassi",
    "Mohamed CHETOUANI",
    "Nathanaël Jarrassé",
    "Nicolas Bredeche",
    "Nicolas Perrin-Gilbert",
    "Nicolas Thome",
    "Olivier Schwander",
    "Olivier Sigaud",
    "Pascal Morin",
    "Pierre Bessière",
    "Sinan Haliyo",
    "Stéphane Doncieux",
    "Alessandra Carbone",
    "Elodie Laine",
    "Martin Weigt",
    "Benoit Semelin",
    "Emeric Bron",
    "Emmanuel Bertin",
    "Françoise Combes",
    "Maryvonne Gerin",
    "Philippe Salomé",
    "Baptiste Cecconi",
    "Philippe Zarka",
    "Ferdinand Dhombres",
    "Jean Charlet",
    "Xavier Tannier",
    "Amal \"El Fallah Seghrouchni\"",
    "Andrea Pinna",
    "Antoine Miné",
    "Béatrice Bérard",
    "Bertrand Granado",
    "Bruno Escoffier",
    "Carola Doerr",
    "Christoph Dürr",
    "Christophe Denis",
    "Christophe Marsala",
    "Colette Faucher",
    "Emmanuel HYON",
    "Emmanuelle Encrenaz-Tiphène",
    "Evripidis Bampis",
    "Fanny Pascual",
    "Haralampos Stratigopoulos",
    "Jean-Daniel Kant",
    "Jean-Gabriel Ganascia",
    "Jean-Noël Vittaut",
    "Lionel Tabourier",
    "Maria Potop-Butucaru",
    "Matthieu Latapy",
    "Nicolas MAUDET",
    "Olivier Spanjaard",
    "Patrice Perny",
    "Patrick Gallinari",
    "Sébastien Tixeuil",
    "Spyros Angelopoulos",
    "Stéphane Gançarski",
    "Vanda Luengo",
    "Vincent Guigue",
    "Bruno Despres",
    "Frédéric Nataf",
    "Julien Brajard",
    "Sylvie Thiria",
    "Catherine Matias",
    "Charlotte Dion-Blanc",
    "Claire Boyer",
    "Gérard Biau",
    "Gregory Nuel",
    "Idris Kharroubi",
    "Maud Thomas",
    "Maxime Sangnier",
    "Olivier Lopez",
    "Sylvain Le-Corff",
    "Tabea Rebafka",
    "Benjamin Fuks",
    "Stéphane Mottelet",
    "Tien-Tuan Dao",
    "julien mozziconacci",
    "Nicolas Aunai",
    "Thierry Dufour",
    "Abdenour Hadid",
    "Benjamin Quost",
    "Bruno Toupance",
    "Dominique Lenne",
    "Evelyne Heyer",
    "Franz Manni",
    "Grace Younes",
    "Lama Tarsissi",
    "Marie-Hélène (Mylène) Masson",
    "Marie-Hélène Abel",
    "Nathalie Martial-Braz",
    "Nicolas Patin",
    "Philippe Bonnifait",
    "Philippe Boulanger",
    "Philippe Trigano",
    "Raed Abu Zitar",
    "Samuel F. Feng",
    "Sébastien Destercke",
    "Tanujit Chakraborty",
    "Yves Grandvalet",
    "Zoheir ABOURA"
]

# Initialisez une liste vide pour stocker les métadonnées des publications
publications = []

# URL de base de l'API HAL
base_url = "https://api.archives-ouvertes.fr/search/"

# Parcourez la liste des auteurs et récupérez leurs publications
for auteur in auteurs:
    params = {
        "q": f'authFullName_s:"{auteur}"',
        "fl": "authFullName_s,authIdHal_i,authIdHal_s,title_s,halId_s,producedDateY_i,doiId_s,abstract_s,uri_s,domain_s,keyword_s",
        "rows": 10000  # Augmentez le nombre de lignes si nécessaire
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        publications.extend(data["response"]["docs"])
    else:
        print(f"Erreur pour l'auteur {auteur}: {response.status_code} - {response.text}")

# Créez un DataFrame Pandas à partir des métadonnées des publications
column_order = ["authFullName_s", "authIdHal_i", "authIdHal_s", "title_s", "halId_s", "producedDateY_i", "doiId_s", "keyword_s", "abstract_s", "uri_s", "domain_s"]

df = pd.DataFrame(publications, columns= column_order)

# Sauvegardez le DataFrame dans un fichier CSV
#df.to_csv("publications_hal_scai_complet.csv", index=False)

# Affichez le nombre total de publications récupérées
print(f"Nombre total de publications récupérées : {len(df)}")

Nombre total de publications récupérées : 12388


In [26]:
# Conversion les listes en chaînes dans la colonne "title_s"
df["title_s"] = df["title_s"].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Suppression des doublons
df = df.drop_duplicates(subset="title_s")
dfp = df.dropna(subset=["abstract_s", "keyword_s"])
print(f"Publications restantes après nettoyage : {len(dfp)}")

Publications restantes après nettoyage : 4206


In [28]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

dfp = dfp.copy()
dfp['language'] = 'unknown'

def detect_language(text):
    try:
        return detect(text)
    except:
        return "UNKNOWN"

def lang(df):
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df.loc[i,'language'] = detect_language(str(df.loc[i,'abstract_s']))
    print("Détection de la langue terminée \n")
    return df

dfp = lang(dfp)

def counter(df):
    counts = df['language'].value_counts()
    print("Nombre d'articles anglais :")
    print(counts.get('en', 0))

    print("Nombre d'articles francais :")
    print(counts.get('fr', 0))

    print("Nombre d'articles allemand :")
    print(counts.get('de', 0))

    print("Index des articles avec langue inconnue")
    print(df[df['language'] == 'UNKNOWN'].index.tolist())
    return

counter(dfp)


Détection de la langue terminée 

Nombre d'articles anglais :
3547
Nombre d'articles francais :
654
Nombre d'articles allemand :
2
Index des articles avec langue inconnue
[]


Cependant, nous remarquons que les métadonnées des publications par HAL ne sont pas aussi complètes que celles d'OpenAlex. 

In [39]:
# ID de l'auteur
author_id = "a5007814380"  

# URL de l'API
url = f"https://api.openalex.org/works?filter=author.id:{author_id}"

# Faire la requête à l'API
response = requests.get(url)

# Vérifier que la requête a réussi
if response.status_code == 200:
    
    # Conversion de la réponse en JSON
    data = json.loads(response.text)
    print(data.keys())
    i=1
    for work in data['results'] : 
        if i==1 :
            print(work.keys())
            i+=1

dict_keys(['meta', 'results', 'group_by'])
dict_keys(['id', 'doi', 'title', 'display_name', 'publication_year', 'publication_date', 'ids', 'language', 'primary_location', 'type', 'type_crossref', 'indexed_in', 'open_access', 'authorships', 'countries_distinct_count', 'institutions_distinct_count', 'corresponding_author_ids', 'corresponding_institution_ids', 'apc_list', 'apc_paid', 'has_fulltext', 'cited_by_count', 'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext', 'primary_topic', 'topics', 'keywords', 'concepts', 'mesh', 'locations_count', 'locations', 'best_oa_location', 'sustainable_development_goals', 'grants', 'datasets', 'versions', 'referenced_works_count', 'referenced_works', 'related_works', 'ngrams_url', 'abstract_inverted_index', 'cited_by_api_url', 'counts_by_year', 'updated_date', 'created_date'])


In [43]:
# ID de l'auteur
author_id = "a5007814380"  

# URL de l'API
url = f"https://api.openalex.org/works?filter=author.id:{author_id}"

# Faire la requête à l'API
response = requests.get(url)

# Vérifier que la requête a réussi
if response.status_code == 200:
    # Conversion de la réponse en JSON
    data = json.loads(response.text)

    # Sélection les métadonnées précises
    selected_data = []
    
    for work in data['results']:
        
        author_ids = []
        if 'authorships' in work: 
            for authorship in work['authorships']:
                # Ajoutez l'identifiant de l'auteur à la liste 'author_ids'
                author_ids.append(authorship["author"]["id"])
                
        selected_data.append({
            'id' : work['id'],
            'title': work['title'],
            'authors_id' : author_ids, 
            'referenced_works': work['referenced_works'], 
            'related_works': work['related_works'], 
            'cited_by_api_url': work['cited_by_api_url'],    
        })

    # Conversion des données sélectionnées en DataFrame
    df = pd.DataFrame(selected_data)

    # Exportation du DataFrame en CSV
    df.to_csv('meta_pub_oa.csv', index=False)
else:
    print(f"Erreur : {response.status_code}")

In [ ]:
auteurs = [
    "a5063079937",
    "a5064456106",
    "a5075692514",
    "a5076657262",
    "a5060263799",
    "a5059667793",
    "a5010298238",
    "a5009962581",
    "a5019206372",
    "a5026117716",
    "Clément Mabi",
    "a5045653236", #David Flacher
    # pas là "M. Shawky",
    "a5054128778", #Serge Bouchardon 
    "a5083701333",
    #Bérangère Bihan-Avalle
    #Caroline Marti
    "a5043441322", #Laurent Petit
    "a5065665642", # ou  Pierre-Carl Langlais a5090355875 
    "a5044456322",
    "a5072965626",
    "a5050309898",
    "a5022276793",
    "a5035893506", #Guilhem Lavaux, y en a 2 donc un dechet
    #"Henry Joy McCracken", chelou
    "a5051397820",
    "a5075521248", # Yohan Dubois en x2
    "a5048168479",
    "a5008261497",
    "a5006372418",
    # existe pas "Slavica Jonic",
    "a5032837587",
    "a5082959305",
    "a5000035181", # y en a 2?? "Pierre-Yves Boëlle",
    "a5008914363", # y en a 2 "Renaud Piarroux",
    "a5012443434",
    "a5086118723", # Jean marc chouvel y en a 3
    "a5042745853",
    "a5085170922",
    "a5073978648",
    "a5060825472",
    "a5009145141", # 2 differents "Benjamin Piwowarski",
    "a5005922530", # 2 a cause du nom "Catherine Achard",
    "a5079026902", # 2 a cause du nom "Catherine Pelachaud",
    "a5019033794", # plusieurs "Gilles Bailly",
    "a5084974660",
    "a5015493311",
    "a5000942708", # 2 differents "Laure Soulier",
    "a5024969402",
    "a5022871131", # 2 differents Matthieu Cord
    "a5016381589",
    "a5049398785",
    "a5057665161", # 2 différents "Nathanaël Jarrassé",
    "a5008211251",
    "a5047587289",
    "a5017490804",
    "a5018812634",
    "a5042850624",
    "a5020385150", # 2 differents importants "Pascal Morin",
    "a5053141872", # 2 differents "Pierre Bessière",
    "a5016542830", # 2 differents "Sinan Haliyo",
    "a5003629424",
    "a5038174177", # 2 differents importants  "Alessandra Carbone" 
    "a5076453514", # 2 differents "Elodie Laine",
    "a5035771024",
    "a5049132515",
    "a5007007631",
    "a5089708177", # doute "Emmanuel Bertin",
    "a5064812526",
    "a5008102443", # 2 differents "Maryvonne Gerin",
    "a5061694871",
    "a5032663932",
    "a5074319502", # 2 differents " Philippe Zarka",
    "a5005420349",
    "a5055383240",
    "a5056834851", # 2 differents " Xavier Tannier",
    "a5044546919", # plusieurs differents importants  " Amal El Fallah-Seghrouchni " 
    "a5019342840",
    "a5069148908",
    "a5008504744",
    "a5057674250",
    "a5020410268",
    "a5040561209",
    "a5029270613",
    "a5003029415",
    "a5076504951", # 2 differents " Christophe Marsala",
    "a5008947610",
    "a5062847773", # 2 differents " Emmanuel Hyon "
    "a5040506059",
    "a5063171222", # 2 differents "Evripidis Bampis "
    "a5031467845",
    "a5091734149",
    "a5020645950", # 2 differents " Jean-Daniel Kant "
    "a5052738299", # 2 differents "Jean‐Gabriel Ganascia "
    "a5009688030",
    "a5056981393",
    "a5080217489", # 2 differents Maria Potop-Butucaru 
    "a5031952531", # 2 differents Matthieu Latapy  
    "a5000925369",
    "a5012670875",
    "a5025612115",
    "a5086752907",
    "a5073883755", # 2 differents Sébastien Tixeuil 
    "a5063338404", # 2 differents Spyros Angelopoulos 
    "a5049035512",
    "a5073711335",
    "a5044389669",
    "a5090273130",
    "a5004798802",
    "a5073300655", # 3 differents Julien Brajard
    "a5042022387", # 2 differents Sylvie Thiria 
    "a5082187550",
    "a5011096724",
    "a5010301554",
    "a5007814380",
    "a5087944743",
    "a5051885232",# 2 differents Idris Kharroubi 
    "a5076190399",# 2 differents Maud Thomas 
    "a5043113193",
    "a5041522495", # 3 differents importants Olivier Lopez
    "a5049226969",
    "a5011379612",
    "a5029271059",
    "a5043406295",
    "a5051516688", #  Tien Tuan Dao  pas de mention SU
    "a5027346778",
    "a5040717345",
    "a5048594819", # 2 differents AThierry Dufour 
    "a5013928164", # 2 differents Abdenour Hadid 
    "a5090882219",
    "a5024260775",
    "a5051963748",
    "a5005868901",
    "a5044064042", # 2 differents  Franz Manni  
    "a5027364763", # 2 differents Grace Younes 
    "a5033136511",
    "a5076437822",
    "a5064018319",
    "a5049226428",
    "a5000802564",
    "a5050418541", # 2 differents Philippe Bonnifait 
    "a5085744367",
    "a5054524844", # pas de mention de SU Philippe Trigano
    "a5078607983", # 3 differents  Raed Abu Zitar 
    "a5008158634",
    "a5070285963",
    "a5012926469",
    "a5021351429",
    "a5009983613"
]
